In [ ]:
# Import required packages and modules
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import parser
import time
import os
import glob

In [ ]:
# Retrieve documents from Federal Registry
# Establish start and end dates for the range of documents
start_date = '2008-08-08'
end_date = '2016-07-01'

# Create date objects for comparisons
start_date_dobj = parser.parse(start_date)
end_date_dobj = parser.parse(end_date)

# Establish a max number of data retrievals
n = 1
# Get data and save each retrieval to a .csv file
while (start_date_dobj < end_date_dobj) and (n < 100):
  print(f"Getting data for: {end_date}")
  print(f"Get attempt: {n}")
  final_url = f'https://www.federalregister.gov/api/v1/documents.csv?fields%5B%5D=abstract&fields%5B%5D=agency_names&fields%5B%5D=publication_date&fields%5B%5D=raw_text_url&fields%5B%5D=title&fields%5B%5D=topics&per_page=1000&conditions%5Bpublication_date%5D%5Bgte%5D={start_date}&conditions%5Bpublication_date%5D%5Blte%5D={end_date}&conditions%5Btype%5D%5B%5D=RULE&conditions%5Btype%5D%5B%5D=PRORULE'
  get_data = pd.read_csv(final_url)
  get_data.to_csv(f'{end_date}.csv')
  end_date_raw = get_data['publication_date'].iloc[-1]
  end_date = parser.parse(end_date_raw).strftime('%Y-%m-%d')
  end_date_dobj = parser.parse(end_date)
  n += 1
  time.sleep(5)

In [ ]:
# Format each file as a .csv
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [ ]:
# Combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# Export the concatenated csv file
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [ ]:
# Read in the combined document data
df = pd.read_csv('combined_csv.csv')
df

,Unnamed: 0,abstract,agency_names,publication_date,raw_text_url,title,topics
0,0,We are adopting a new airworthiness directive ...,Transportation Department; Federal Aviation Ad...,09/02/2015,https://www.federalregister.gov/documents/full...,Airworthiness Directives; Airbus Airplanes,Air transportation; Aircraft; Aviation safety;...
1,1,We are adopting a new airworthiness directive ...,Transportation Department; Federal Aviation Ad...,09/02/2015,https://www.federalregister.gov/documents/full...,"Airworthiness Directives; Bombardier, Inc. Air...",Air transportation; Aircraft; Aviation safety;...
2,2,We are adopting a new airworthiness directive ...,Transportation Department; Federal Aviation Ad...,09/02/2015,https://www.federalregister.gov/documents/full...,Airworthiness Directives; The Boeing Company A...,Air transportation; Aircraft; Aviation safety;...
3,3,We are adopting a new airworthiness directive ...,Transportation Department; Federal Aviation Ad...,09/02/2015,https://www.federalregister.gov/documents/full...,Airworthiness Directives; Lockheed Martin Corp...,Air transportation; Aircraft; Aviation safety;...
4,4,We are adopting a new airworthiness directive ...,Transportation Department; Federal Aviation Ad...,09/02/2015,https://www.federalregister.gov/documents/full...,Airworthiness Directives; Vulcanair S.p.A. Air...,Air transportation; Aircraft; Aviation safety;...
...,...,...,...,...,...,...,...
48569,995,This airworthiness directive (AD) supersedes t...,Transportation Department; Federal Aviation Ad...,03/24/2009,https://www.federalregister.gov/documents/full...,Airworthiness Directives; Airbus Model A310 Se...,Air transportation; Aircraft; Aviation safety;...
48570,996,We are adopting a new airworthiness directive ...,Transportation Department; Federal Aviation Ad...,03/24/2009,https://www.federalregister.gov/documents/full...,Airworthiness Directives; Empresa Brasileira d...,Air transportation; Aircraft; Aviation safety;...
48571,997,We are adopting a new airworthiness directive ...,Transportation Department; Federal Aviation Ad...,03/24/2009,https://www.federalregister.gov/documents/full...,Airworthiness Directives; Boeing Model 757-200...,Air transportation; Aircraft; Aviation safety;...
48572,998,The FAA is superseding an existing airworthine...,Transportation Department; Federal Aviation Ad...,03/24/2009,https://www.federalregister.gov/documents/full...,Airworthiness Directives; Fokker Model F.27 Ma...,Air transportation; Aircraft; Aviation safety;...


In [ ]:
# Take a look at the indexing. The Unnamed:0 column starts over at 1000. I'll need to reindex the data
df.loc[1998]

Unnamed: 0                                                        998
abstract            This action announces the Environmental Protec...
agency_names                          Environmental Protection Agency
publication_date                                           06/08/2010
raw_text_url        https://www.federalregister.gov/documents/full...
title               Expedited Approval of Alternative Test Procedu...
topics              Chemicals; Environmental protection; Indians-l...
Name: 1998, dtype: object

In [ ]:
# Need to look at missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48574 entries, 0 to 48573
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        48574 non-null  int64 
 1   abstract          47828 non-null  object
 2   agency_names      48549 non-null  object
 3   publication_date  48574 non-null  object
 4   raw_text_url      48574 non-null  object
 5   title             48574 non-null  object
 6   topics            34379 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.6+ MB


In [ ]:
# Missing values
df.isna().sum()

Unnamed: 0              0
abstract              746
agency_names           25
publication_date        0
raw_text_url            0
title                   0
topics              14195
dtype: int64

In [ ]:
# Determine percentage of missing abstracts
abs_na = df['abstract']
x = abs_na.isna().sum()
print(round((x / len(df)) * 100, 2),'%')

1.54 %


In [ ]:
# Set the publication date to a datetime object
df['publication_date'] = pd.to_datetime(df['publication_date'])

In [ ]:
# Sort the dataframe by publication date
df = df.sort_values(by=['publication_date'])
df.head()

,Unnamed: 0,abstract,agency_names,publication_date,raw_text_url,title,topics
29572,572,This final rule updates the payment rates used...,Health and Human Services Department; Centers ...,2008-08-08,https://www.federalregister.gov/documents/full...,Medicare Program; Prospective Payment System a...,Health facilities; Kidney diseases; Medicare; ...
29571,571,This final rule updates the prospective paymen...,Health and Human Services Department; Centers ...,2008-08-08,https://www.federalregister.gov/documents/full...,Medicare Program; Inpatient Rehabilitation Fac...,Administrative practice and procedure; Health ...
29556,556,This final rule revises and clarifies requirem...,Agriculture Department; Food and Nutrition Ser...,2008-08-08,https://www.federalregister.gov/documents/full...,Management of Donated Foods in Child Nutrition...,Administrative practice and procedure; Food as...
29557,557,The Board is publishing a final rule amending ...,Federal Reserve System,2008-08-08,https://www.federalregister.gov/documents/full...,Truth in Lending,Advertising; Federal Reserve System; Mortgages...
29558,558,"The Commander, First Coast Guard District, has...",Homeland Security Department; Coast Guard,2008-08-08,https://www.federalregister.gov/documents/full...,Drawbridge Operation Regulations; Shrewsbury R...,NaN


In [ ]:
# The csv concatenation process added an unneeded column. Drop the 'Unnamed: 0' column as it has no purpose.
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head(1)

,abstract,agency_names,publication_date,raw_text_url,title,topics
29572,This final rule updates the payment rates used...,Health and Human Services Department; Centers ...,2008-08-08,https://www.federalregister.gov/documents/full...,Medicare Program; Prospective Payment System a...,Health facilities; Kidney diseases; Medicare; ...


In [ ]:
# Reset the index after sorting the data by date
df = df.reset_index(drop=True)
df.head()

,abstract,agency_names,publication_date,raw_text_url,title,topics
0,This final rule updates the payment rates used...,Health and Human Services Department; Centers ...,2008-08-08,https://www.federalregister.gov/documents/full...,Medicare Program; Prospective Payment System a...,Health facilities; Kidney diseases; Medicare; ...
1,This final rule updates the prospective paymen...,Health and Human Services Department; Centers ...,2008-08-08,https://www.federalregister.gov/documents/full...,Medicare Program; Inpatient Rehabilitation Fac...,Administrative practice and procedure; Health ...
2,This final rule revises and clarifies requirem...,Agriculture Department; Food and Nutrition Ser...,2008-08-08,https://www.federalregister.gov/documents/full...,Management of Donated Foods in Child Nutrition...,Administrative practice and procedure; Food as...
3,The Board is publishing a final rule amending ...,Federal Reserve System,2008-08-08,https://www.federalregister.gov/documents/full...,Truth in Lending,Advertising; Federal Reserve System; Mortgages...
4,"The Commander, First Coast Guard District, has...",Homeland Security Department; Coast Guard,2008-08-08,https://www.federalregister.gov/documents/full...,Drawbridge Operation Regulations; Shrewsbury R...,NaN


In [ ]:
# Print the columns
cols = df.columns.tolist()
cols

['abstract',
 'agency_names',
 'publication_date',
 'raw_text_url',
 'title',
 'topics']

In [ ]:
# Reorder the columns using a list

cols_reorder = ['publication_date', 'abstract', 'title', 'topics', 'agency_names', 'raw_text_url']
df = df[cols_reorder]
df.head(1)

,publication_date,abstract,title,topics,agency_names,raw_text_url
0,2008-08-08,This final rule updates the payment rates used...,Medicare Program; Prospective Payment System a...,Health facilities; Kidney diseases; Medicare; ...,Health and Human Services Department; Centers ...,https://www.federalregister.gov/documents/full...


In [ ]:
# Rename the columns using a dictionary
cols_rename = {'publication_date':'date', 'abstract':'abstract', 'title':'title',
               'topics':'topics', 'agency_names':'agency_names', 'raw_text_url':'raw_text_url'}
df.rename(columns = cols_rename, inplace=True)
df.head(1)

,date,abstract,title,topics,agency_names,raw_text_url
0,2008-08-08,This final rule updates the payment rates used...,Medicare Program; Prospective Payment System a...,Health facilities; Kidney diseases; Medicare; ...,Health and Human Services Department; Centers ...,https://www.federalregister.gov/documents/full...


In [ ]:
# Saving the current dataframe to a csv file.
df.to_csv('fed_reg_docs_8yr.csv', index=False)

In [ ]:
# Daily DJIA stock data same 5-yr period as the government documents data.
df_djia = pd.read_csv('Combined_News_DJIA.csv')
df_djia

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",b'Georgian troops retreat from S. Osettain cap...,b'Did the U.S. Prep Georgia for War with Russia?',b'Rice Gives Green Light for Israel to Attack ...,b'Announcing:Class Action Lawsuit on Behalf of...,"b""So---Russia and Georgia are at war and the N...","b""China tells Bush to stay out of other countr...",b'Did World War III start today?',b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,b'Welcome To World War IV! Now In High Definit...,"b""Georgia's move, a mistake of monumental prop...",b'Russia presses deeper into Georgia; U.S. say...,b'Abhinav Bindra wins first ever Individual Ol...,b' U.S. ship heads for Arctic to define territ...,b'Drivers in a Jerusalem taxi station threaten...,b'The French Team is Stunned by Phelps and the...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...","b""The US military was surprised by the timing ...",b'U.S. Beats War Drum as Iran Dumps the Dollar',"b'Gorbachev: ""Georgian military attacked the S...",b'CNN use footage of Tskhinvali ruins to cover...,b'Beginning a war as the Olympics were opening...,b'55 pyramids as large as the Luxor stacked in...,b'The 11 Top Party Cities in the World',b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who k

In [ ]:
df_djia.drop(df_djia.iloc[:, 2:], inplace = True, axis = 1)
df_djia.head(1)

,Date,Label
0,2008-08-08,0


In [ ]:
# Renaming columns
djia_cols_rename = {'Date':'date', 'Label':'label'}
df_djia.rename(columns = djia_cols_rename, inplace=True)
df_djia

,date,label
0,2008-08-08,0
1,2008-08-11,1
2,2008-08-12,0
3,2008-08-13,0
4,2008-08-14,1
...,...,...
1984,2016-06-27,0
1985,2016-06-28,1
1986,2016-06-29,1
1987,2016-06-30,1


In [ ]:
# Converting "date" column to pandas datetime
df_djia['date'] = pd.to_datetime(df_djia['date'])
df_djia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1989 non-null   datetime64[ns]
 1   label   1989 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 31.2 KB


In [ ]:
# Daily DJIA stock data same 5-yr period as the government documents data.
df_djia_raw = pd.read_csv('upload_DJIA_table.csv')
df_djia_raw

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234
...,...,...,...,...,...,...,...
1984,2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688
1985,2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961
1986,2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727
1987,2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609


In [ ]:
# Renaming columns
djia_raw_rename = {'Date':'date', 'Open': 'open', 'High':'high', 'Low':'low',
                    'Close':'close', 'Volume':'volume', 'Adj Close':'adj_close'}
df_djia_raw.rename(columns = djia_raw_rename, inplace=True)
df_djia_raw.head(1)

,date,open,high,low,close,volume,adj_close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141


In [ ]:
# Converting "date" column to pandas datetime
df_djia_raw['date'] = pd.to_datetime(df_djia_raw['date'])
df_djia_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       1989 non-null   datetime64[ns]
 1   open       1989 non-null   float64       
 2   high       1989 non-null   float64       
 3   low        1989 non-null   float64       
 4   close      1989 non-null   float64       
 5   volume     1989 non-null   int64         
 6   adj_close  1989 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 108.9 KB


In [ ]:
# Merge the df_djia to the df_djia_raw
df_djia_all = pd.merge(df_djia, df_djia_raw, how='left', on='date')
df_djia_all.head(1)

,date,label,open,high,low,close,volume,adj_close
0,2008-08-08,0,11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312


In [ ]:
df_djia_all

,date,label,open,high,low,close,volume,adj_close
0,2008-08-08,0,11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312
1,2008-08-11,1,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609
2,2008-08-12,0,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727
3,2008-08-13,0,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961
4,2008-08-14,1,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688
...,...,...,...,...,...,...,...,...
1984,2016-06-27,0,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234
1985,2016-06-28,1,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
1986,2016-06-29,1,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
1987,2016-06-30,1,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234


In [ ]:
# Saving the df_djia_all dataframe to a csv file.
df_djia_all.to_csv('djia_all_8yr.csv', index = False)